In [ ]:
from bsb import from_storage


In [ ]:
scaffold = from_storage('network.hdf5')

In [ ]:
config = scaffold.configuration
print(f"My network was configured with {config}")
# scaffold.cell_types corresponds to scaffold.configuration.cell_types
print(f"My network has {len(scaffold.cell_types)} cell types")

In [ ]:
import numpy as np

In [ ]:
for type_name, cell_type in scaffold.cell_types.items():
    ps = cell_type.get_placement_set()
    pos = ps.load_positions()
    print(f"{len(pos)} {type_name} placed")
    # The positions are a (Nx3) numpy array
    print("The median cell is located at", np.median(pos, axis=0))

In [ ]:
cs = scaffold.get_connectivity_set("A_to_B")
# Retrieve connection pairs
for src_locs, dest_locs in cs.load_connections():
    # 0: neuron id
    # 1: neuron branch
    # 2: neuron point
    print(f"Cell id: {src_locs[0]} connects to cell {dest_locs[0]}")

In [ ]:
from matplotlib import pylab as plt

In [ ]:
radii=[]
cell_names=[]

for type_name, cell_type in scaffold.cell_types.items():
    radii.append(cell_type.spatial.radius)
    cell_names.append(type_name)

In [ ]:
cell_names

In [ ]:
radii

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax1 = fig.add_subplot(111, projection='3d')

color_map = plt.cm.jet([0.25,0.75])

for i, cell_type in enumerate(cell_names):
    ps = scaffold.get_placement_set(cell_type)
    positions = ps.load_positions()
    ax1.scatter(positions[:, 0], 
                    positions[:, 1], 
                    positions[:, 2], 
                    c=color_map[np.where(np.array(cell_names)==cell_type)[0]],
                    label=cell_type,
                    s=radii[i]
                    )
ax1.set_xlabel("x in $\mu m$")
ax1.set_ylabel("y in $\mu m$")
ax1.set_zlabel("z in $\mu m$")
ax1.legend()

plt.title("Placement results", fontsize=40)
plt.tight_layout()
    

In [ ]:
import pandas as pd

In [ ]:
#Loop through tge entire connectivity set
dset = pd.DataFrame(columns=["Connection name", "Nb. Synapses", "Synapses per pair", "Convergence", "Divergence"])
for ps in scaffold.get_connectivity_sets():
    #Get the ConnectivityIterator for the current connectivity strategy
    cs = scaffold.get_connectivity_set(ps.tag).load_connections().as_globals()
    pre_locs, post_locs = cs.all()
    # Find the pairs of pre-post neurons (combos) and count how many synapses there are between each pair (combo_counts)
    combos, combo_counts = np.unique(np.column_stack((pre_locs[:, 0], post_locs[:, 0])), axis=0, return_counts=True)
    
    # Find the unique post and pre neurons
    uniquePre, uniquePre_count = np.unique(combos[:,0], axis=0, return_counts=True)
    niquePost, uniquePost_count = np.unique(combos[:,1], axis=0, return_counts=True)
    
    dset.loc[len(dset)] = [ps.tag, len(pre_locs), 
                           "{:.2} $\pm$ {:.2}".format(np.mean(combo_counts), np.std(combo_counts)),
                           "{:.2} $\pm$ {:.2}".format(np.mean(uniquePost_count), np.std(uniquePost_count)),
                           "{:.2} $\pm$ {:.2}".format(np.mean(uniquePre_count), np.std(uniquePre_count)),
                          ]
dset